In [ ]:
#import driver and other libraries


from selenium.webdriver import Chrome
import pandas as pd
import time
from selenium.webdriver.common.keys import Keys
#import java.util.concurrent.TimeUnit;

webdriver = "C:/Users/sdam/Documents/IESEG/Financial Programming Python/Project/chromedriver_win32/chromedriver.exe"

driver = Chrome(webdriver)
#driver.manage().timeouts().implicitlyWait(10, TimeUnit.SECONDS);
url = "https://www.bienici.com/recherche/achat/tourcoing-59200,roubaix-59100?tri=surface-desc" 
driver.get(url)
time.sleep(20)

python_button = driver.find_elements_by_xpath('//*[@id="searchSideHeader"]/div/div[2]/div/div/div/div[2]')[0]
python_button.click()

quotes = driver.find_elements_by_class_name("sideListItemSecondBlock")
quote = quotes[2]

Title = quote.find_element_by_class_name('descriptionTitle')
try:
    Title.click()
except :
    pass

time.sleep(5)
    
length = driver.find_element_by_class_name('totalDetailedSheets')
len=int(length.text)



In [ ]:
#navigate webpages in the driver


datalist = []
for x in range(1,len+1):
    time.sleep(10)
    try:
        #Handle pop-up ads in any page
        
        ads = driver.find_element_by_xpath('//*[@id="loader"]/div[6]/div/div/div/div/button')
        ads.send_keys(Keys.RETURN)
    except:
        pass
    time.sleep(7)
    
    label = driver.find_element_by_class_name('realEstateAdsMainInfo').text
    address= driver.find_element_by_class_name('fullAddress').text
    
     
    
    #following are put in try loops because sometimes they are not there
    try: 
        energy1 = driver.find_element_by_xpath('/html/body/div[1]/section/div/div[3]/section[4]/div[2]/div[1]/div[1]/span').text
        #energy1 = Energy Performance indicator
       
    except:
        energy1 = "NA"
    
    try:
        energy2 = driver.find_element_by_xpath('/html/body/div[1]/section/div/div[3]/section[4]/div[2]/div[2]/div[1]/span').text
        # energy2 = Greenhouse gas emissions (both are yearly estimation)
    except:
        energy2 = "NA"
    
    try:
        energy1label = driver.find_element_by_xpath('/html/body/div[1]/section/div/div[3]/section[4]/div[2]/div[1]/div[1]/span[2]').text
    except:
        energy1label ="NA"
   
    try: 
        energy2label = driver.find_element_by_xpath('/html/body/div[1]/section/div/div[3]/section[4]/div[2]/div[2]/div[1]/span[2]').text
    except:
        energy2label ="NA"
    
    time.sleep(4)
    try:
        pricesquare = driver.find_element_by_xpath('/html/body/div[1]/section/div/div[3]/div[1]/div[1]/div[2]/div[1]/div/div/div/span[2]').text
        
    except:
        pricesquare = "NA" 
    
    specifics = driver.find_elements_by_class_name('labelInfo')
    informationlist= []
    for i in specifics:
        x = i.text
        informationlist.append(x)
    
    button= driver.find_element_by_id('selectNextDetailedSheet')
    time.sleep(2)
    try:
        button.click()
    except :
        pass
    
 #putting all the information into a usable dataframe   
        
    info=((label,address,energy1,energy1label,energy2,energy2label,pricesquare,informationlist))
    datalist.append(info)
    dataframe = pd.DataFrame(datalist,columns=['label','adress','energy1','energy1label','energy2','energy2label','pricesquare','informationlist'])
    
 
    
driver.close() 


In [ ]:
dataframe.to_csv("Tourcoing_Roubaix.csv")

In [ ]:
dataframe_copy2 = dataframe.copy()
dataframe_copy2.head()

In [ ]:
#splitting label dateofpublishment and modified/referencenr 
#I use two steps to overcome the problem that sometimes there is no "modifiée"
splitted = dataframe_copy2["label"].str.split("|",n=2,expand=True)
dataframe_copy2["reference"]=splitted[2]
dataframe_copy2["modified"]=splitted[1]
dataframe_copy2["published"]=splitted[0]
dataframe_copy2.drop(columns =["label"], inplace= True)


#replace None value with the value of modified and replacing modified with None if it contained reference number
for x in range(1,len+1):
    if dataframe_copy2.reference[x] == None:
        dataframe_copy2.reference[x] = dataframe_copy2.modified[x]

        
dataframe_copy2['modified'][dataframe_copy2['modified'].str.contains('Réf',na = False)] = None

#splitting adress
splitted = dataframe_copy2["adress"].str.split("(",n=1,expand=True)
dataframe_copy2["municipality"]=splitted[0]
dataframe_copy2["adressdescription"]=splitted[1]


splitted = dataframe_copy2["municipality"].str.split(" ",n=1,expand=True)
dataframe_copy2["municipality"]=splitted[0]
dataframe_copy2["postalcode"]=splitted[1]

dataframe_copy2['adressdescription']= dataframe_copy2['adressdescription'].str.replace(")"," ", regex=True)
dataframe_copy2.drop(columns =["adress"], inplace= True)


#extracting referencenumber from reference
splitted = dataframe_copy2["reference"].str.split(": ",n=1,expand=True)
dataframe_copy2["reference"]=splitted[1]



#cleaning both dates
splitted = dataframe_copy2["modified"].str.split("le ",n=1,expand=True)
dataframe_copy2["modified"]=splitted[1]
dataframe_copy2.head()

#removing the dot and replacing é's by e (putting it as a proper date)
dataframe_copy2['modified']= dataframe_copy2['modified'].str.replace("é","e", regex=True)
dataframe_copy2['modified']= dataframe_copy2['modified'].str.replace("."," ", regex=True)
splitted = dataframe_copy2["published"].str.split("le ",n=1,expand=True)
dataframe_copy2["published"]=splitted[1]
dataframe_copy2["published"]= dataframe_copy2["published"].str.replace("é","e", regex=True)
dataframe_copy2["published"]= dataframe_copy2["published"].str.replace("."," ", regex=True)


#removing the €/m² to allows for easier graphs
dataframe_copy2["euro/m²"]= dataframe_copy2["pricesquare"].str.replace("€/m²"," ", regex=True)
dataframe_copy2.drop(columns =["pricesquare"], inplace= True)
dataframe_copy2["euro/m²"]= dataframe_copy2["euro/m²"].str.replace(" ","", regex=True)
dataframe_copy2["euro/m²"].loc[219]= None
dataframe_copy2["euro/m²"]= dataframe_copy2["euro/m²"].str.replace("NA","", regex=True)
dataframe_copy2["euro/m²"].fillna(0)
dataframe_copy2["euro/m²"]=pd.to_numeric(dataframe_copy2["euro/m²"])


#Creation of a new dataframe_copy2 splitted where we are going to create variables out of
dataframe_copy2['informationlist']=dataframe_copy2['informationlist'].astype(str)
dataframe_copy2['informationlist']=dataframe_copy2['informationlist'].fillna('')

splitted = dataframe_copy2["informationlist"].str.split(",",expand=True)

#creation of variable price
splitted[0]= splitted[0].str.replace("[","", regex=True)
splitted[0]= splitted[0].str.replace(" €","", regex=True)
splitted[0]= splitted[0].str.replace("\\","", regex=True)
splitted[0]= splitted[0].str.replace("'","", regex=True)
splitted[0]= splitted[0].str.replace("Prixu202f:","", regex=True)

#we're only using the minimum price in case of multiple prices for the sake of simplicity
intermediate = splitted[0].str.split(" à",n=1,expand=True)
splitted[0]=intermediate[0]
dataframe_copy2["price"]=splitted[0]

dataframe_copy2["price"]= dataframe_copy2["price"].str.replace(" ","", regex=True)
dataframe_copy2["price"][219] = 0
dataframe_copy2["price"][257] = 0
dataframe_copy2["price"]=pd.to_numeric(dataframe_copy2["price"])

#creating of variable honoraires
#loop that searches for a value and then assigns a value which i can reuse
import numpy as np
dataframe_copy2["honoraires"] = np.nan * splitted.shape[0]


for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        
        if splitted.iloc[row,col] == None:
            continue
        if ('honoraires :' in splitted.iloc[row,col].casefold()) or ('honoraires à la charge du vendeur' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["honoraires"][row] = splitted.iloc[row,col]

            
#cleaning honoraires
dataframe_copy2['honoraires']= dataframe_copy2['honoraires'].str.replace("honoraires :","", regex=True)
dataframe_copy2['honoraires']= dataframe_copy2['honoraires'].str.replace("'","", regex=True)
#this is solving a random mistake that slipped into the variable
#dataframe_copy2['honoraires']= dataframe_copy2['honoraires'].str.replace("12 000 € TTC inclus à la charge de l’acquéreur","NaN", regex=True)


#creation of variable heating system
dataframe_copy2["heating"] = np.nan *  splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('chauffage' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["heating"][row] = splitted.iloc[row,col]

#cleaning heating
dataframe_copy2['heating']= dataframe_copy2['heating'].str.replace("'","", regex=True)
dataframe_copy2['heating']= dataframe_copy2['heating'].str.replace("Chauffage : ","", regex=True)

#creation of variable rooms
dataframe_copy2["rooms"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('chambre' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["rooms"][row] = splitted.iloc[row,col]
#cleaning the rooms variable
dataframe_copy2["rooms"]= dataframe_copy2["rooms"].str.replace("'","", regex=True)
dataframe_copy2["rooms"]= dataframe_copy2["rooms"].str.replace(" chambres","", regex=True)
dataframe_copy2["rooms"]= dataframe_copy2["rooms"].str.replace(" chambre","", regex=True)

#creation of variable pièces
dataframe_copy2["pieces"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('pièce' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["pieces"][row] = splitted.iloc[row,col]

#cleaning pieces
dataframe_copy2["pieces"]= dataframe_copy2["pieces"].str.replace("'","", regex=True)
dataframe_copy2["pieces"]= dataframe_copy2["pieces"].str.replace("pièces","", regex=True)
dataframe_copy2["pieces"]= dataframe_copy2["pieces"].str.replace("pièce","", regex=True)

#creation of dummy garden
dataframe_copy2["dummygarden"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('jardin' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["dummygarden"][row] = int(1)
            
#replacing NAN with 0
dataframe_copy2["dummygarden"].fillna(0)

#creation of variable bathrooms
dataframe_copy2["bathroom"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('de bain' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["bathroom"][row] = splitted.iloc[row,col]
            
#cleaning bathrooms
dataframe_copy2["bathroom"]= dataframe_copy2["bathroom"].str.replace("'","", regex=True)
dataframe_copy2["bathroom"]= dataframe_copy2["bathroom"].str.replace("salle de bain","", regex=True)
dataframe_copy2["bathroom"]= dataframe_copy2["bathroom"].str.replace("salles de bain","", regex=True)
            
#creation of dummy terrace
dataframe_copy2["dummyterrace"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('balcon' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["dummyterrace"][row] = 1
            
#replacing NAN with 0
dataframe_copy2["dummyterrace"].fillna(0)

#creation of dummy cellar
dataframe_copy2["dummycellar"] = np.nan * splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('cave' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["dummycellar"][row] = 1

#replacing NAN with 0
dataframe_copy2["dummycellar"].fillna(0)
            
#creation of dummy parking
dataframe_copy2["dummyparking"] = np.nan *  splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('parking' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["dummyparking"][row] = 1

#replacing NAN with 0
dataframe_copy2["dummyparking"].fillna(0)
            
#creation of dummy elevator
dataframe_copy2["dummyelevator"] = np.nan *  splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('ascenseur' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["dummyelevator"][row] = 1
            
#replacing NAN with 0
dataframe_copy2["dummyelevator"].fillna(0)
            
#creation of variable living surface
dataframe_copy2["livingsurface"] = np.nan *  splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ("m²'" in splitted.iloc[row,col].casefold()):
            dataframe_copy2["livingsurface"][row] = splitted.iloc[row,col]
            
            

            
for row in range(dataframe_copy2["livingsurface"].shape[0]):
    try:
        if ("Balcon" in dataframe_copy2["livingsurface"][row]) or ("Jardin" in dataframe_copy2["livingsurface"][row]):
            dataframe_copy2["livingsurface"][row]= np.nan
    except:
        continue
#clean livingsurface
dataframe_copy2["livingsurface"]= dataframe_copy2["livingsurface"].str.replace("'","", regex=True)
dataframe_copy2["livingsurface"]= dataframe_copy2["livingsurface"].str.replace(" m²","", regex=True)
        
#creation of totalsurface
dataframe_copy2["totalsurface"] = np.nan *  splitted.shape[0]

for col in range(splitted.columns.shape[0]):
    for row in range(splitted[col].shape[0]):
        if splitted.iloc[row,col] == None:
            continue
        if ('de terrain' in splitted.iloc[row,col].casefold()):
            dataframe_copy2["totalsurface"][row] = splitted.iloc[row,col]
dataframe_copy2.drop(columns =["informationlist"], inplace= True)

#clean totalsurface
dataframe_copy2["totalsurface"]= dataframe_copy2["totalsurface"].str.replace("'","", regex=True)
dataframe_copy2["totalsurface"]= dataframe_copy2["totalsurface"].str.replace(" m² de terrain","", regex=True)




In [ ]:
dataframe_copy2["rooms"] = pd.to_numeric(dataframe_copy2["rooms"])
dataframe_copy2["price"] = pd.to_numeric(dataframe_copy2["price"])
dataframe_copy2["euro/m²"]= pd.to_numeric(dataframe_copy2["euro/m²"])
dataframe_copy2["livingsurface"] = pd.to_numeric(dataframe_copy2["livingsurface"])

splitted = dataframe_copy2["pieces"].str.rsplit(" ",n=5,expand=True)
dataframe_copy2["pieces"] = pd.to_numeric(splitted[1])

In [ ]:
dataframe_copy2['totalsurface']= dataframe_copy2["totalsurface"].str.replace(" ","", regex=True)
dataframe_copy2['totalsurface']= pd.to_numeric(dataframe_copy2["totalsurface"])
dataframe_copy2['municipality'] = dataframe_copy2['municipality'].astype('category')

In [ ]:
dataframe_copy3 = dataframe_copy2.copy()
dataframe_copy3.to_csv("Tourcoing_Roubaix_clean1.csv")

# Visualization

In [2]:
pip install bokeh

Note: you may need to restart the kernel to use updated packages.


In [3]:
from bokeh.plotting import figure, output_file, show, save, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.transform import factor_cmap,cumsum
from bokeh.palettes import Blues8,Category20c
from bokeh.embed import components
from bokeh.models import CategoricalColorMapper,Slider,Select
from bokeh.layouts import row, column,widgetbox
from bokeh.models.widgets import Tabs,Panel
from bokeh.io import curdoc,push_notebook, show, output_notebook

import pandas as pd

In [4]:
import numpy as np

In [5]:
dataframe_copy3 = pd.read_csv(r"C:\Users\sdam\Documents\IESEG\Financial Programming Python\Project\Presentation\Tourcoing_Roubaix_clean.csv").drop(['Unnamed: 0'],axis=1)

dataframe_copy3.head()

,energy1,energy1label,energy2,energy2label,reference,modified,published,municipality,adressdescription,postalcode,...,rooms,pieces,dummygarden,bathroom,dummyterrace,dummycellar,dummyparking,dummyelevator,livingsurface,totalsurface
0,Vierge,NaN,Vierge,NaN,GB00092048,21 nov 2019,NaN,Tourcoing,Blanche Porte - Flocon,59200,...,NaN,1,NaN,NaN,NaN,1.0,1.0,NaN,397,NaN
1,143,C,25,D,VNMW1668,30 nov 2019,NaN,Roubaix,Fresnoy - Mackellerie,59100,...,6.0,9,1.0,NaN,NaN,1.0,1.0,NaN,350,1000.0
2,267,E,53,E,VM23257-IMMOTRAM2,5 dec 2019,NaN,Roubaix,NaN,59100,...,7.0,9,1.0,2.0,NaN,1.0,1.0,NaN,330,731.0
3,134,C,31,D,ROUB-350k-13857,30 nov 2019,NaN,Roubaix,Sainte-Elisabeth,59100,...,5.0,9,NaN,NaN,NaN,NaN,NaN,NaN,330,NaN
4,189,D,44,E,SO1641,23 oct 2019,NaN,Roubaix,Barbieux - Edouard Vaillant - Vauban,59100,...,7.0,9,1.0,2.0,NaN,1.0,NaN,NaN,300,650.0


In [6]:
energy1labeldata = dataframe_copy3.groupby(['energy1label','municipality'],as_index=False).count()
energy2labeldata = dataframe_copy3.groupby(['energy2label','municipality'],as_index=False).count()
energy1labeldata.head()

,energy1label,municipality,energy1,energy2,energy2label,reference,modified,published,adressdescription,postalcode,...,rooms,pieces,dummygarden,bathroom,dummyterrace,dummycellar,dummyparking,dummyelevator,livingsurface,totalsurface
0,A,Roubaix,3,7,3,7,2,5,1,7,...,7,7,2,6,0,1,0,0,7,2
1,A,Tourcoing,1,3,1,3,1,3,2,3,...,3,3,0,2,0,0,1,1,3,0
2,B,Roubaix,2,2,2,2,2,1,2,2,...,2,2,1,2,0,1,0,0,2,1
3,B,Tourcoing,5,5,5,5,4,2,4,5,...,5,5,1,5,3,0,5,4,5,1
4,C,Roubaix,26,27,26,27,23,8,20,27,...,24,27,15,17,0,7,2,3,27,15


In [7]:
color_list1 = energy1labeldata['municipality'].unique()
color_list2 = energy2labeldata['municipality'].unique()



In [8]:



# Create ColumnDataSource from data frame
source1 = ColumnDataSource(energy1labeldata)
source2 = ColumnDataSource(energy2labeldata)

output_file('index.html')



labels_list1 = energy1labeldata['energy1label'].unique().tolist()
labels_list2 = energy2labeldata['energy2label'].unique().tolist()


# Add plot
p1 = figure(
    y_range=labels_list1,
    plot_width=500,
    plot_height=500,
    title='Houses available for puchase by Energy Rating 1',
    x_axis_label='Count',
    tools="pan,box_select,zoom_in,zoom_out,save,reset"
)

# Render glyph
p1.hbar(
    y='energy1label',
    right='price',
    left=0,
    height=0.4,
    fill_color=factor_cmap(
      'municipality',
      palette= ['#deebf7','#3182bd'],
      factors=color_list1
    ),
    fill_alpha=0.9,
    source=source1,
    legend='municipality'
)

# Add Legend
p1.legend.orientation = 'vertical'
p1.legend.location = 'top_right'
p1.legend.label_text_font_size = '10px'

# Add Tooltips
hover = HoverTool(tooltips=[('count:','@reference' ),('Locality:','@municipality' )])

p1.add_tools(hover)

# Add plot
p2 = figure(
    y_range=labels_list2,
    plot_width=500,
    plot_height=500,
    title='Houses available for puchase by Energy Rating 2',
    x_axis_label='Count',
    tools="pan,box_select,zoom_in,zoom_out,save,reset"
)

# Render glyph
p2.hbar(
    y='energy2label',
    right='price',
    left=0,
    height=0.4,
    fill_color=factor_cmap(
      'municipality',
      palette= ['#deebf7','#3182bd'],
      factors=color_list2
    ),
    fill_alpha=0.9,
    source=source2,
    legend='municipality'
)

# Add Legend
p2.legend.orientation = 'vertical'
p2.legend.location = 'top_right'
p2.legend.label_text_font_size = '10px'


p2.add_tools(hover)

tab1 = Panel(child=p1, title='Availability as per Energy Labels 1')
tab2 = Panel(child=p2, title='Availability as per Energy Labels 2')

#layout = row(p1, p2)
layout = Tabs(tabs=[tab1, tab2])



# Show results
show(layout)


In [9]:

# slider = Slider(title='my slider', start=0, end=10, step=0.1, value=2)
# # Create a widgetbox layout: layout
# layout = widgetbox(slider)

# # Add the layout to the current document
# curdoc().add_root(layout)
